In [10]:
!pip install numpy pandas scikit-learn xgboost tensorflow

python(89125) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [3]:
import pandas as pd
import polars as pl 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

For ada solana and eth: Long

In [8]:
data_ada_long = pd.read_csv('Users/jadenfix/downloads/processed_data_ada_long.csv')
print("Ada long:",data_ada_long.head())

data_solana_long = pd.read_csv('Users/jadenfix/downloads/processed_data_solana_long.csv')
print("Solana long:",data_solana_long.head())

data_eth_long = pd.read_csv('Users/jadenfix/downloads/processed_data_eth_long.csv')
print("ETH long:",data_eth_long.head())


Ada long:    eth_open  eth_high  eth_low  eth_close  eth_volume  ada_open  ada_high  \
0    377.74    378.24   377.43     377.94  1066.72997   0.13618   0.13639   
1    377.92    378.17   377.88     377.95   399.58109   0.13618   0.13629   
2    377.95    378.71   377.88     378.65   686.35004   0.13617   0.13629   
3    378.65    379.77   378.65     379.75  1301.39604   0.13626   0.13668   
4    379.77    379.93   379.69     379.85   319.49589   0.13671   0.13684   

   ada_low  ada_close  ada_volume  ...  solana_close_lag3  \
0  0.13606    0.13622    397998.4  ...             3.2985   
1  0.13609    0.13620    182810.7  ...             3.2815   
2  0.13595    0.13622    386508.3  ...             3.2690   
3  0.13626    0.13666    198676.4  ...             3.2676   
4  0.13670    0.13672    128888.1  ...             3.2538   

                    timestamp   ada_log     ada_r  ada_r_next  ada_r_winsor  \
0  2020-08-12T00:03:00.000000 -1.993484 -0.000440   -0.000147     -0.000440   
1 

For short: ada btc eth and solana
/Users/jadenfix/Downloads/processed_data_solana.csv /Users/jadenfix/Downloads/processed_data_ada.csv /Users/jadenfix/Downloads/processed_data_eth.csv /Users/jadenfix/Downloads/processed_data_btc.csv

In [9]:
data_ada_short = pd.read_csv('Users/jadenfix/downloads/processed_data_ada.csv')
print("Ada short:",data_ada_short.head())
data_solana_short = pd.read_csv('Users/jadenfix/downloads/processed_data_solana.csv')
print("Solana short:",data_solana_short.head())
data_eth_short = pd.read_csv('Users/jadenfix/downloads/processed_data_eth.csv')
print("ETH short:",data_eth_short.head())
data_btc_short = pd.read_csv('Users/jadenfix/downloads/processed_data_btc.csv')
print("BTC short:",data_btc_short.head())

Ada short:    btc_open  btc_high   btc_low  btc_close  btc_volume  eth_open  eth_high  \
0  29050.29  29050.29  29050.29   29050.29         0.0   1829.91   1830.50   
1  29050.29  29050.29  29050.29   29050.29         0.0   1830.49   1830.68   
2  29050.29  29050.29  29050.29   29050.29         0.0   1830.67   1830.76   
3  29050.29  29050.29  29050.29   29050.29         0.0   1830.76   1830.93   
4  29050.29  29050.29  29050.29   29050.29         0.0   1830.92   1831.17   

   eth_low  eth_close  eth_volume  ...  solana_close_lag3  \
0  1829.53    1830.49    362.8306  ...              22.79   
1  1830.49    1830.68     29.8070  ...              22.77   
2  1830.67    1830.75     28.3587  ...              22.77   
3  1830.75    1830.92     14.4738  ...              22.77   
4  1830.92    1831.17     82.1231  ...              22.78   

                    timestamp   ada_log     ada_r  ada_r_next  ada_r_winsor  \
0  2023-08-05T00:03:00.000000 -1.225878  0.000000    0.000341      0.00000

In [16]:
# crypto_time_series_forecast.py
# ---------------------------------------
# Time-Series Forecasting of Crypto Returns
# Models: PINN, CNN, RNN, MLP, BayesianRidge, XGBoost, AdaBoost, Stacked Ensemble
# Uses TimeSeriesSplit for robust time-series cross-validation on one-minute data

import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import AdaBoostRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential, callbacks

# 1) LOAD DATA
paths = {
    'ada':   '/Users/jadenfix/downloads/processed_data_ada.csv',
    'solana':'/Users/jadenfix/downloads/processed_data_solana.csv',
    'eth':   '/Users/jadenfix/downloads/processed_data_eth.csv',
    'btc':   '/Users/jadenfix/downloads/processed_data_btc.csv'
}
# parse timestamp
data = {coin: pd.read_csv(path, parse_dates=['timestamp']) for coin, path in paths.items()}

# 2) UTIL FUNCTIONS

def metrics(y_true, y_pred):
    # Adds MAPE for percent error insight
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    with np.errstate(divide='ignore', invalid='ignore'):
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return {'mae': mae, 'rmse': rmse, 'r2': r2, 'mape': mape}

# Generic TS CV for sklearn-style models
def cv_sklearn_model(model, X, y, n_splits=3):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    for train_idx, test_idx in tscv.split(X):
        X_tr, X_te = X[train_idx], X[test_idx]
        y_tr, y_te = y[train_idx], y[test_idx]
        model.fit(X_tr, y_tr)
        preds = model.predict(X_te)
        fold_metrics.append(metrics(y_te, preds))
    return {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}

# Prepare features/target
def prepare_xy(df, coin):
    df = df.sort_values('timestamp').dropna()
    X = df.drop(columns=[f'{coin}_r_next', 'timestamp']).values
    y = df[f'{coin}_r_next'].values
    return X, y

# 3) SKLEARN MODEL FUNCTIONS

def bayesian_ridge_cv(X, y):
    return cv_sklearn_model(BayesianRidge(), X, y)


def xgboost_cv(X, y):
    model = xgb.XGBRegressor(
        n_estimators=50,
        max_depth=3,
        learning_rate=0.1,
        objective='reg:squarederror',
        verbosity=0
    )
    return cv_sklearn_model(model, X, y)


def adaboost_cv(X, y):
    base = DecisionTreeRegressor(max_depth=3)
    model = AdaBoostRegressor(
        estimator=base,
        n_estimators=30,
        learning_rate=0.1,
        random_state=42
    )
    return cv_sklearn_model(model, X, y)

# 4) NEURAL-NET MODELS

# MLP builder and CV
def build_mlp(n_features):
    return Sequential([
        layers.Input(shape=(n_features,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])

def mlp_cv(X, y, n_splits=3, epochs=10, batch_size=256):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    for tr_idx, te_idx in tscv.split(X):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler = StandardScaler().fit(X_tr)
        X_tr_s = scaler.transform(X_tr)
        X_te_s = scaler.transform(X_te)
        model = build_mlp(X_tr_s.shape[1])
        model.compile(optimizer='adam', loss='mse')
        model.fit(
            X_tr_s, y_tr,
            validation_data=(X_te_s, y_te),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)],
            verbose=0
        )
        preds = model.predict(X_te_s).flatten()
        fold_metrics.append(metrics(y_te, preds))
        tf.keras.backend.clear_session()
    return {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}

# CNN builder and CV
def build_cnn(n_features):
    return Sequential([
        layers.Input(shape=(n_features, 1)),
        layers.Conv1D(32, kernel_size=3, activation='relu'),
        layers.Flatten(),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])

def cnn_cv(X, y, n_splits=3, epochs=10, batch_size=256):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    for tr_idx, te_idx in tscv.split(X):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler = StandardScaler().fit(X_tr)
        X_tr_s = scaler.transform(X_tr)[..., np.newaxis]
        X_te_s = scaler.transform(X_te)[..., np.newaxis]
        model = build_cnn(X_tr.shape[1])
        model.compile(optimizer='adam', loss='mse')
        model.fit(
            X_tr_s, y_tr,
            validation_data=(X_te_s, y_te),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)],
            verbose=0
        )
        preds = model.predict(X_te_s).flatten()
        fold_metrics.append(metrics(y_te, preds))
        tf.keras.backend.clear_session()
    return {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}

# RNN builder and CV
def build_rnn(n_features):
    return Sequential([
        layers.Input(shape=(1, n_features)),
        layers.SimpleRNN(32, activation='tanh'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])

def rnn_cv(X, y, n_splits=3, epochs=10, batch_size=256):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    for tr_idx, te_idx in tscv.split(X):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler = StandardScaler().fit(X_tr)
        X_tr_s = scaler.transform(X_tr).reshape(-1, 1, X_tr.shape[1])
        X_te_s = scaler.transform(X_te).reshape(-1, 1, X_te.shape[1])
        model = build_rnn(X_tr.shape[1])
        model.compile(optimizer='adam', loss='mse')
        model.fit(
            X_tr_s, y_tr,
            validation_data=(X_te_s, y_te),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)],
            verbose=0
        )
        preds = model.predict(X_te_s).flatten()
        fold_metrics.append(metrics(y_te, preds))
        tf.keras.backend.clear_session()
    return {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}

# 5) PINN: Physics-Informed NN enforcing output smoothness
class PINN(Model):
    def __init__(self, n_features, alpha=1.0):
        super().__init__()
        self.d1 = layers.Dense(64, activation='tanh')
        self.d2 = layers.Dense(64, activation='tanh')
        self.out = layers.Dense(1)
        self.alpha = alpha
    def call(self, x):
        z = self.d1(x)
        z = self.d2(z)
        return self.out(z)
    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            data_loss = tf.reduce_mean(tf.square(y - y_pred))
            with tf.GradientTape() as g2:
                g2.watch(x)
                y_hat = self(x)
            dy_dx = g2.gradient(y_hat, x)
            phys_loss = tf.reduce_mean(tf.square(dy_dx))
            loss = data_loss + self.alpha * phys_loss
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return {'loss': loss, 'data_loss': data_loss, 'phys_loss': phys_loss}

def pinn_cv(X, y, n_splits=3, epochs=10, batch_size=256, alpha=1.0):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    for tr_idx, te_idx in tscv.split(X):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler = StandardScaler().fit(X_tr)
        X_tr_s = scaler.transform(X_tr)
        X_te_s = scaler.transform(X_te)
        model = PINN(X_tr.shape[1], alpha)
        model.compile(optimizer='adam')
        model.fit(X_tr_s, y_tr, epochs=epochs, batch_size=batch_size, verbose=0)
        preds = model.predict(X_te_s).flatten()
        fold_metrics.append(metrics(y_te, preds))
        tf.keras.backend.clear_session()
    return {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}

# 6) Stacked Ensemble

def stacked_cv(X, y, n_splits=3):
    estimators = [
        ('bayes', BayesianRidge()),
        ('xgb', xgb.XGBRegressor(n_estimators=30, max_depth=3, learning_rate=0.1, objective='reg:squarederror', verbosity=0)),
        ('ada', AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=3), n_estimators=20, learning_rate=0.1, random_state=42))
    ]
    stack = StackingRegressor(estimators=estimators, final_estimator=BayesianRidge())
    return cv_sklearn_model(stack, X, y)

# 7) RUN ALL MODELS

def run_all(data_dict):
    records = []
    for coin, df in data_dict.items():
        X, y = prepare_xy(df, coin)
        records.append({'coin': coin, 'model': 'BayesianRidge', **bayesian_ridge_cv(X, y)})
        records.append({'coin': coin, 'model': 'XGBoost', **xgboost_cv(X, y)})
        records.append({'coin': coin, 'model': 'AdaBoost', **adaboost_cv(X, y)})
        records.append({'coin': coin, 'model': 'MLP', **mlp_cv(X, y)})
        records.append({'coin': coin, 'model': 'CNN', **cnn_cv(X, y)})
        records.append({'coin': coin, 'model': 'RNN', **rnn_cv(X, y)})
        records.append({'coin': coin, 'model': 'PINN', **pinn_cv(X, y)})
        records.append({'coin': coin, 'model': 'Stacked', **stacked_cv(X, y)})
    return pd.DataFrame(records)

# Example usage
if __name__ == '__main__':
    results = run_all(data)
    print(results)


1677/1677 ━━━━━━━━━━━━━━━━━━━━ 2s 964us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 726us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 453us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 343us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 292us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 403us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 298us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 0s 251us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 0s 277us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 448us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 458us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 494us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 505us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 562us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 482us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 284us/step
1677/1677 ━━━━━━━━━━━━

In [ ]:
# crypto_time_series_forecast.py
# ---------------------------------------
# Time-Series Forecasting of Crypto Returns
# Models: PINN, CNN, RNN, MLP, BayesianRidge, XGBoost, AdaBoost, Stacked Ensemble
# Uses TimeSeriesSplit for robust time-series cross-validation on one-minute data

import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import AdaBoostRegressor, StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential, callbacks

# 1) LOAD DATA
paths = {
    'ada':   '/Users/jadenfix/downloads/processed_data_ada.csv',
    'solana':'/Users/jadenfix/downloads/processed_data_solana.csv',
    'eth':   '/Users/jadenfix/downloads/processed_data_eth.csv',
    'btc':   '/Users/jadenfix/downloads/processed_data_btc.csv'
}
data = {coin: pd.read_csv(path, parse_dates=['timestamp']) for coin, path in paths.items()}

# 2) UTIL FUNCTIONS

def metrics(y_true, y_pred):
    """
    Compute regression metrics including MAPE with safe handling of zero targets.
    """
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    # Avoid division by zero in MAPE
    mask = np.abs(y_true) > 1e-8
    if np.any(mask):
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    else:
        mape = np.nan
    return {'mae': mae, 'rmse': rmse, 'r2': r2, 'mape': mape}

# Generic TimeSeries CV for sklearn-style models

def cv_sklearn_model(model, X, y, n_splits=3):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    for train_idx, test_idx in tscv.split(X):
        X_tr, X_te = X[train_idx], X[test_idx]
        y_tr, y_te = y[train_idx], y[test_idx]
        model.fit(X_tr, y_tr)
        preds = model.predict(X_te)
        fold_metrics.append(metrics(y_te, preds))
    return {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}

# Prepare features/target
def prepare_xy(df, coin):
    df = df.sort_values('timestamp').dropna()
    X = df.drop(columns=[f'{coin}_r_next', 'timestamp']).values
    y = df[f'{coin}_r_next'].values
    return X, y

# 3) SKLEARN MODEL FUNCTIONS

def bayesian_ridge_cv(X, y):
    return cv_sklearn_model(BayesianRidge(), X, y)


def xgboost_cv(X, y):
    model = xgb.XGBRegressor(
        n_estimators=50,
        max_depth=3,
        learning_rate=0.1,
        objective='reg:squarederror',
        verbosity=0
    )
    return cv_sklearn_model(model, X, y)


def adaboost_cv(X, y):
    base = DecisionTreeRegressor(max_depth=3)
    model = AdaBoostRegressor(
        estimator=base,
        n_estimators=30,
        learning_rate=0.1,
        random_state=42
    )
    return cv_sklearn_model(model, X, y)

# 4) NEURAL NET MODELS

# MLP

def build_mlp(n_features):
    return Sequential([
        layers.Input(shape=(n_features,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])

def mlp_cv(X, y, n_splits=3, epochs=10, batch_size=256):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    for tr_idx, te_idx in tscv.split(X):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler = StandardScaler().fit(X_tr)
        X_tr_s = scaler.transform(X_tr)
        X_te_s = scaler.transform(X_te)
        model = build_mlp(X_tr_s.shape[1])
        model.compile(optimizer='adam', loss='mse')
        model.fit(
            X_tr_s, y_tr,
            validation_data=(X_te_s, y_te),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)],
            verbose=0
        )
        preds = model.predict(X_te_s).flatten()
        fold_metrics.append(metrics(y_te, preds))
        tf.keras.backend.clear_session()
    return {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}

# CNN

def build_cnn(n_features):
    return Sequential([
        layers.Input(shape=(n_features, 1)),
        layers.Conv1D(32, kernel_size=3, activation='relu'),
        layers.Flatten(),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])

def cnn_cv(X, y, n_splits=3, epochs=10, batch_size=256):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    for tr_idx, te_idx in tscv.split(X):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler = StandardScaler().fit(X_tr)
        X_tr_s = scaler.transform(X_tr)[..., np.newaxis]
        X_te_s = scaler.transform(X_te)[..., np.newaxis]
        model = build_cnn(X_tr.shape[1])
        model.compile(optimizer='adam', loss='mse')
        model.fit(
            X_tr_s, y_tr,
            validation_data=(X_te_s, y_te),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)],
            verbose=0
        )
        preds = model.predict(X_te_s).flatten()
        fold_metrics.append(metrics(y_te, preds))
        tf.keras.backend.clear_session()
    return {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}

# RNN

def build_rnn(n_features):
    return Sequential([
        layers.Input(shape=(1, n_features)),
        layers.SimpleRNN(32, activation='tanh'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])

def rnn_cv(X, y, n_splits=3, epochs=10, batch_size=256):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    for tr_idx, te_idx in tscv.split(X):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler = StandardScaler().fit(X_tr)
        X_tr_s = scaler.transform(X_tr).reshape(-1, 1, X_tr.shape[1])
        X_te_s = scaler.transform(X_te).reshape(-1, 1, X_te.shape[1])
        model = build_rnn(X_tr.shape[1])
        model.compile(optimizer='adam', loss='mse')
        model.fit(
            X_tr_s, y_tr,
            validation_data=(X_te_s, y_te),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[callbacks.EarlyStopping(patience=2, restore_best_weights=True)],
            verbose=0
        )
        preds = model.predict(X_te_s).flatten()
        fold_metrics.append(metrics(y_te, preds))
        tf.keras.backend.clear_session()
    return {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}

# 5) PINN: Physics-Informed NN enforcing smoothness
class PINN(Model):
    def __init__(self, n_features, alpha=1.0):
        super().__init__()
        self.d1 = layers.Dense(64, activation='tanh')
        self.d2 = layers.Dense(64, activation='tanh')
        self.out = layers.Dense(1)
        self.alpha = alpha
    def call(self, x):
        z = self.d1(x)
        z = self.d2(z)
        return self.out(z)
    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            data_loss = tf.reduce_mean(tf.square(y - y_pred))
            with tf.GradientTape() as g2:
                g2.watch(x)
                y_hat = self(x)
            dy_dx = g2.gradient(y_hat, x)
            phys_loss = tf.reduce_mean(tf.square(dy_dx))
            loss = data_loss + self.alpha * phys_loss
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return {'loss': loss, 'data_loss': data_loss, 'phys_loss': phys_loss}

def pinn_cv(X, y, n_splits=3, epochs=10, batch_size=256, alpha=1.0):
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    for tr_idx, te_idx in tscv.split(X):
        X_tr, X_te = X[tr_idx], X[te_idx]
        y_tr, y_te = y[tr_idx], y[te_idx]
        scaler = StandardScaler().fit(X_tr)
        X_tr_s = scaler.transform(X_tr)
        X_te_s = scaler.transform(X_te)
        model = PINN(X_tr.shape[1], alpha)
        model.compile(optimizer='adam')
        model.fit(X_tr_s, y_tr, epochs=epochs, batch_size=batch_size, verbose=0)
        preds = model.predict(X_te_s).flatten()
        fold_metrics.append(metrics(y_te, preds))
        tf.keras.backend.clear_session()
    return {k: np.mean([fm[k] for fm in fold_metrics]) for k in fold_metrics[0]}

# 6) Stacked Ensemble
def stacked_cv(X, y, n_splits=3):
    estimators = [
        ('bayes', BayesianRidge()),
        ('xgb', xgb.XGBRegressor(n_estimators=30, max_depth=3, learning_rate=0.1, objective='reg:squarederror', verbosity=0)),
        ('ada', AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=3), n_estimators=20, learning_rate=0.1, random_state=42))
    ]
    stack = StackingRegressor(estimators=estimators, final_estimator=BayesianRidge())
    return cv_sklearn_model(stack, X, y)

# 7) RUN ALL MODELS
def run_all(data_dict):
    records = []
    for coin, df in data_dict.items():
        X, y = prepare_xy(df, coin)
        records.append({'coin': coin, 'model': 'BayesianRidge', **bayesian_ridge_cv(X, y)})
        records.append({'coin': coin, 'model': 'XGBoost', **xgboost_cv(X, y)})
        records.append({'coin': coin, 'model': 'AdaBoost', **adaboost_cv(X, y)})
        records.append({'coin': coin, 'model': 'MLP', **mlp_cv(X, y)})
        records.append({'coin': coin, 'model': 'CNN', **cnn_cv(X, y)})
        records.append({'coin': coin, 'model': 'RNN', **rnn_cv(X, y)})
        records.append({'coin': coin, 'model': 'PINN', **pinn_cv(X, y)})
        records.append({'coin': coin, 'model': 'Stacked', **stacked_cv(X, y)})
    return pd.DataFrame(records)

# Example usage
if __name__ == '__main__':
    results = run_all(data)
    print(results)


1677/1677 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 487us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 495us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 644us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 461us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 465us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 398us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 2s 605us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 333us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 2s 908us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 345us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 567us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 467us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 483us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 2s 905us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 671us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 578us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 1s 373us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 2s 910us/step
1677/1677 ━━━━━━━━━━━━━━━━━━━━ 0s 238us/step
1677/1677 ━━━━━━